Question:
- How many times do people view the item before purchase.
- what time and what day have the highest traffic of people viewing products
- what date have the most sales and least sales.
- How loyal are people to a brand. (How often the same individual repurchases an items from the same brand.)
- When the person purchased the item, what other item did the person also purchase at the same time? (using the association rules)

In [10]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/krits/Documents/GitHub/ICCS361Project1/DATA/2019-Oct.csv')
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


1000    False
1001    False
1002    False
1003    False
1004    False
        ...  
1995     True
1996    False
1997    False
1998    False
1999     True
Name: user_session, Length: 1000, dtype: bool

Total number of duplicate rows: 30220
Number of duplicates in each column:
event_time       39827226
event_type       42448761
product_id       42281970
category_id      42448140
category_code    42448637
brand            42445319
price            42383466
user_id          39426474
user_session     33204342
dtype: int64


In [9]:
# Question 1: When the person purchased the item, what other item did the person also purchase at the same time?
# We will filter the dataframe for 'purchase' events and group by 'user_session' to see other items purchased in the same session.

purchase_df = df[df['event_type'] == 'purchase']
grouped_purchase = purchase_df.groupby('user_session')['product_id'].apply(list).reset_index()
grouped_purchase.columns = ['user_session', 'purchased_items']
print(grouped_purchase.head())

# Question 2: How loyal are people to a brand. (How often the same individual come back to buy products from a certain brand)
# We will group by 'user_id' and 'brand' to count the number of purchases per user per brand.

loyalty_df = purchase_df.groupby(['user_id', 'brand']).size().reset_index(name='purchase_count')
print(loyalty_df.head())

# Question 3: When do people usually buy stuff/items
# We will extract the hour from 'event_time' and count the number of purchases per hour.

df['event_time'] = pd.to_datetime(df['event_time'])
purchase_df['event_time'] = pd.to_datetime(purchase_df['event_time'])  # Ensure datetime conversion
purchase_df['hour'] = purchase_df['event_time'].dt.hour
hourly_purchases = purchase_df.groupby('hour').size().reset_index(name='purchase_count')
print(hourly_purchases)

# Question 4: What time do people view the most? How often is a viewed item purchased? How many people are viewing at that current time.
# We will extract the hour from 'event_time' and count the number of views per hour.
# We will also calculate the conversion rate from view to purchase.

view_df = df[df['event_type'] == 'view']
view_df['hour'] = view_df['event_time'].dt.hour
hourly_views = view_df.groupby('hour').size().reset_index(name='view_count')
print(hourly_views)

# Conversion rate from view to purchase
conversion_rate = purchase_df.shape[0] / view_df.shape[0]
print(f"Conversion rate from view to purchase: {conversion_rate:.2%}")

# Question 5: How many times do people view the item before purchase.
# We will group by 'user_id' and 'product_id' to count the number of views before a purchase.

view_counts = view_df.groupby(['user_id', 'product_id']).size().reset_index(name='view_count')
purchase_counts = purchase_df.groupby(['user_id', 'product_id']).size().reset_index(name='purchase_count')
merged_counts = pd.merge(view_counts, purchase_counts, on=['user_id', 'product_id'], how='left')
merged_counts['purchase_count'] = merged_counts['purchase_count'].fillna(0)
print(merged_counts.head())

                           user_session     purchased_items
0  00000083-8816-4d58-a9b8-f52f54186edc           [1004870]
1  000001fd-1f89-45e8-a3ce-fe3218cabfad           [1004834]
2  00004ada-8f93-49a6-956d-4ed71ae94791  [1005031, 1005031]
3  00005026-a9d1-4e2b-8290-3cc14e4bad89           [4000169]
4  00005b76-13ba-4afe-b80d-2f2b337d3e92  [1004806, 1005066]
     user_id     brand  purchase_count
0  264649825  kiturami               2
1  303160429    garmin               1
2  340041246        lg               4
3  384989212     artel               1
4  396222093  cordiant               1


/var/folders/hv/0mg1x66j16ldl69jbflkvgvm0000gn/T/ipykernel_42240/740231827.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_time'] = pd.to_datetime(purchase_df['event_time'])  # Ensure datetime conversion
/var/folders/hv/0mg1x66j16ldl69jbflkvgvm0000gn/T/ipykernel_42240/740231827.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['hour'] = purchase_df['event_time'].dt.hour


    hour  purchase_count
0      0            2949
1      1            5526
2      2           13968
3      3           29634
4      4           41146
5      5           48071
6      6           52005
7      7           53406
8      8           55197
9      9           55183
10    10           51904
11    11           47311
12    12           41888
13    13           39138
14    14           37363
15    15           37632
16    16           35773
17    17           30873
18    18           24325
19    19           16748
20    20            9892
21    21            6278
22    22            3837
23    23            2802


/var/folders/hv/0mg1x66j16ldl69jbflkvgvm0000gn/T/ipykernel_42240/740231827.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view_df['hour'] = view_df['event_time'].dt.hour


    hour  view_count
0      0      299874
1      1      545742
2      2     1037315
3      3     1485926
4      4     1826454
5      5     2020538
6      6     2154468
7      7     2217746
8      8     2267899
9      9     2229264
10    10     2181560
11    11     2086826
12    12     2040927
13    13     2263632
14    14     2590385
15    15     2892601
16    16     2972082
17    17     2663340
18    18     2069720
19    19     1304238
20    20      722821
21    21      425189
22    22      264826
23    23      216026
Conversion rate from view to purchase: 1.82%
     user_id  product_id  view_count  purchase_count
0   33869381     7002639           1             0.0
1   64078358    10600284           1             0.0
2  183503497    22200103           1             0.0
3  184265397     6902133           2             0.0
4  184265397     6902303           2             0.0
